# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

In [2]:
#!pip install gmaps

In [3]:
#!jupyter nbextension enable --py --sys-prefix gmaps

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
JAZweatherpull = "weatherdata.csv"

Vacationstart_df = pd.read_csv(JAZweatherpull)

Vacationstart_df = Vacationstart_df.drop(columns = ['Unnamed: 0'])

weatherdata = Vacationstart_df
weatherdata.head(10)


,City,Country,Temperature,Latitude,Longitude,Max_Temp,Cloudiness,Wind_Speed,Humidity
0,Vao,NC,73.15,-22.67,167.48,73.15,100,11.52,78
1,Ugoofaaru,MV,84.07,5.67,73.00,84.07,100,15.90,80
2,Ust'-Omchug,RU,36.63,61.15,149.63,36.63,99,2.59,83
3,Uglich,RU,40.89,57.53,38.33,40.89,100,2.75,97
4,Cap Malheureux,MU,69.01,-19.98,57.61,69.01,0,1.99,83
5,Punta Arenas,CL,42.39,-53.15,-70.92,42.80,90,10.29,93
6,Rocha,UY,56.16,-34.48,-54.33,56.16,67,10.60,59
7,Busselton,AU,60.03,-33.65,115.33,61.00,99,36.26,67
8,Corona,US,83.43,33.88,-117.57,87.80,1,8.05,42
9,Hermanus,ZA,59.00,-34.42,19.23,59.00,98,5.99,87


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
import gmaps.datasets
gmaps.configure(api_key= g_key)

In [7]:
#Map locations data
locations = weatherdata[['Latitude','Longitude']]

#Map Humidity data
humidwater = weatherdata['Humidity'].astype(float)

humidmax = weatherdata['Humidity'].max()

heatmap_layer = gmaps.heatmap_layer(locations, weights = humidwater, dissipating = False,
                                    max_intensity = humidmax, point_radius =5)

fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
weatherdata.columns

Index(['City', 'Country', 'Temperature', 'Latitude', 'Longitude', 'Max_Temp',
       'Cloudiness', 'Wind_Speed', 'Humidity'],
      dtype='object')

In [9]:
City_goodweather = weatherdata.loc[(weatherdata['Cloudiness']== 0) &
                                   (weatherdata['Max_Temp'] < 80) &
                                    (weatherdata['Max_Temp'] > 70)&
                                   (weatherdata['Wind_Speed'] < 10)]
            
City_goodweather.head(10)

,City,Country,Temperature,Latitude,Longitude,Max_Temp,Cloudiness,Wind_Speed,Humidity
79,East London,ZA,71.40,-33.02,27.91,71.40,0,9.48,33
87,Mossamedes,AO,74.05,-15.20,12.15,74.05,0,5.75,68
162,São Filipe,CV,74.97,14.90,-24.50,74.97,0,5.70,68
249,Saryagash,KZ,71.60,41.45,69.17,71.60,0,6.93,64
280,Esil,KZ,73.65,51.96,66.40,73.65,0,8.97,47
333,Itarema,BR,79.02,-2.92,-39.92,79.02,0,7.38,87
349,Mingora,PK,71.82,34.78,72.36,71.82,0,6.76,37
413,Mineros,BO,73.40,-17.12,-63.23,73.40,0,2.24,33
418,San Fernando del Valle de Catamarca,AR,71.94,-28.47,-65.79,73.99,0,1.01,45
456,Santa Cruz de la Sierra,BO,72.28,-17.80,-63.17,73.40,0,4.65,30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = City_goodweather

hotel_df["Hotel Name"] = ""

hotel_df = hotel_df[['City','Country','Latitude','Longitude','Hotel Name']]
hotel_df


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Latitude,Longitude,Hotel Name
79,East London,ZA,-33.02,27.91,
87,Mossamedes,AO,-15.20,12.15,
162,São Filipe,CV,14.90,-24.50,
249,Saryagash,KZ,41.45,69.17,
280,Esil,KZ,51.96,66.40,
333,Itarema,BR,-2.92,-39.92,
349,Mingora,PK,34.78,72.36,
413,Mineros,BO,-17.12,-63.23,
418,San Fernando del Valle de Catamarca,AR,-28.47,-65.79,
456,Santa Cruz de la Sierra,BO,-17.80,-63.17,


In [11]:
target_coordinates = f"{hotel_df['Latitude']},{hotel_df['Longitude']}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)
#---------

In [12]:
hotel_name = []

# params dictionary to update each iteration
params = {"radius": 5000,"types": "hotel","keyword": "hotel","key": g_key}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{latitude},{longitude}"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))
    
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
        
    except IndexError:
        hotel_name.append(np.nan)
        
len(hotel_name)

13

In [13]:
hotel_df['Hotel Name'] = hotel_name
hotel_df

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Country,Latitude,Longitude,Hotel Name
79,East London,ZA,-33.02,27.91,Lighthouse Accommodation
87,Mossamedes,AO,-15.20,12.15,Hotel ibis Styles Iu Namibe
162,São Filipe,CV,14.90,-24.50,Hotel Xaguate
249,Saryagash,KZ,41.45,69.17,NaN
280,Esil,KZ,51.96,66.40,Egorkino Hotel
333,Itarema,BR,-2.92,-39.92,Hotel do David
349,Mingora,PK,34.78,72.36,Swat View Hotel
413,Mineros,BO,-17.12,-63.23,Residencial Imperial
418,San Fernando del Valle de Catamarca,AR,-28.47,-65.79,Amerian Catamarca Park Hotel
456,Santa Cruz de la Sierra,BO,-17.80,-63.17,Los Tajibos Hotel & Convention Center


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotels = hotel_df[["Latitude", "Longitude"]]


In [20]:
# Add marker layer ontop of heat map

viewmarker = gmaps.marker_layer(hotels, info_box_content = hotel_info,display_info_box = True)

#fig = gmaps.figure()
fig.add_layer(viewmarker)
fig



Figure(layout=FigureLayout(height='420px'))